In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt


In [ ]:
caminho = os.path.join('..', '..', 'db_softtek') 
path_arq = os.path.abspath(caminho)

In [ ]:
tbl_anl = pd.read_excel(path_arq+'\\cubo_anl\\tbl_full_analitica.xlsx')

In [ ]:
#verificando se há valores nulos no data set:

tbl_anl.isnull().sum()

In [ ]:
tbl_anl.dtypes

In [ ]:
# tipando campos necessários:

tbl_anl['VALOR CONTRATO'] = tbl_anl['VALOR CONTRATO'].astype(float)

In [ ]:
tbl_anl.describe()

In [ ]:
## Regras de negócio:

# 1 - Melhorar eficiencia em horas do volume de tratativas
# 2 - Melhorar a distribuição de demandas de acordo com o capacity de cada operador
# 3 - Melhorar a relação de custos de acordo com o capacity

In [ ]:

tbl_anl.HORAS.plot(kind='box')
plt.title('Boxplot para a variavel horas')
plt.ylabel('tempo em horas')
plt.show()

In [ ]:
# Solução para variável horas: Dividir em grupos de forma categorizada


# Função que verificaa o intervalo e retornar a marcação correspondente:

def verifica_intervalo(valor):
    if 0 < valor <= 2:
        return 'MAIOR que 0 e MENOR OU IGUAL a 2' # media
    elif 2 < valor <= 4:
        return 'MAIOR que 2 e MENOR OU IGUAL a 4' # media superior
    elif 4 < valor <= 6:
        return 'MAIOR que 4 e MENOR OU IGUAL a 6' # cauda
    elif valor > 6:
        return 'MAIOR que 6' # Outliers
    else:
        return 'fora dos intervalos' # dados invalidos


tbl_anl['intervalo_horas']= tbl_anl['HORAS'].apply(verifica_intervalo)

In [ ]:
# validação da marcação:

tbl_anl[['HORAS','intervalo_horas']]

In [ ]:
tbl_anl.VALOR_AT.plot(kind='hist',bins=10)
plt.axvline(tbl_anl.VALOR_AT.quantile(q=0.025), c='r')
plt.axvline(tbl_anl.VALOR_AT.quantile(q=0.975), c='r')    
plt.title('Histograma da variavel de Custos')

In [ ]:
## Conclusão do histograma de valores: 

# Frequencia de valores acima de 97,5% é muito pequena (pode ser tratado de outra forma no modelo)

In [ ]:
tbl_consultor = tbl_anl[tbl_anl['STATUS_CHAMADO'] == 'Closed']
tbl_consultor = tbl_consultor.groupby(['IS','MODULO_CHAMADO','TIPO_CHAMADO','MODULO_CustosNSULTOR','COMPLEXIDADE','SENIORIDADE']).agg(
    {'HORAS':'min','VALOR_AT':'min','CHAMADO':'count'}).reset_index()

In [ ]:
tbl_consultor2 = tbl_consultor.groupby(['TIPO_CHAMADO','MODULO_CHAMADO','COMPLEXIDADE','IS','SENIORIDADE']).agg({'CHAMADO':'mean','HORAS':'min',
                                                                                         'VALOR_AT':'min'}).reset_index().sort_values(
                                                                                             ['TIPO_CHAMADO', 'MODULO_CHAMADO','COMPLEXIDADE',
                                                                                             'CHAMADO','HORAS','VALOR_AT'],
                                                                                               ascending=[True,True, True, False,True,True]
                                                                                               )
tbl_consultor2

In [ ]:
# Normalizando as métricas de forma linear ponderada

tbl_consultor2['volume_normalizado'] = tbl_consultor2['CHAMADO'] / tbl_consultor2['CHAMADO'].max()

# Invertendo a normalização para que menor valor de horas por chamado seja melhor
tbl_consultor2['horas_normalizado'] = 1 - (tbl_consultor2['HORAS'] / tbl_consultor2['HORAS'].max())

# Invertendo a normalização para que menor valor de custo por chamado seja melhor
tbl_consultor2['custo_normalizado'] = 1 - (tbl_consultor2['VALOR_AT'] / tbl_consultor2['VALOR_AT'].max())

In [ ]:
# Definindo os pesos para cada métrica:

peso_volume = 0.5
peso_horas = 0.3
peso_custo = 0.2

In [ ]:
# Calculando o score de eficiência combinando os valores ponderados
tbl_consultor2['score_eficiencia'] = (
    (tbl_consultor2['volume_normalizado'] * peso_volume) + 
    (tbl_consultor2['horas_normalizado'] * peso_horas) + 
    (tbl_consultor2['custo_normalizado'] * peso_custo)
)

In [ ]:
tbl_consultor3 = tbl_consultor2[['TIPO_CHAMADO', 'MODULO_CHAMADO','COMPLEXIDADE','IS','score_eficiencia','CHAMADO','SENIORIDADE']]
tbl_consultor3['rank'] = tbl_consultor3.groupby(['TIPO_CHAMADO', 'MODULO_CHAMADO','COMPLEXIDADE'])['score_eficiencia'].rank(ascending=False, method='dense')
tbl_consultor3.sort_values(['TIPO_CHAMADO', 'MODULO_CHAMADO','COMPLEXIDADE','rank'], inplace=True)

tbl_consultor3.head(50)

In [ ]:
''' Saída do modelo de score: '''

tbl_consultor4 = tbl_consultor3.groupby(['TIPO_CHAMADO','MODULO_CHAMADO','COMPLEXIDADE','IS']).agg({'rank':'min','score_eficiencia':'max'}).reset_index().sort_values(['TIPO_CHAMADO', 'MODULO_CHAMADO','COMPLEXIDADE','IS','rank']).drop_duplicates(subset='IS',keep='first')
tbl_consultor4.to_excel(path_arq+'\\modelo\\Modelo_score_train.xlsx',index=False)

In [ ]:
tbl_consultor4.sort_values(['TIPO_CHAMADO','MODULO_CHAMADO','COMPLEXIDADE','rank'])